In [1]:
import vk

In [2]:
class ConfigAPI(object):
    __token = None
    api = None

    #Стартовый обработчик. Подключает TOKEN VK API к приложению
    def init(self):
        print('Вставьте ссылку в строку браузера для получения токена VK API. ')
        print('https://oauth.vk.com/authorize?client_id=7305133&display=page&scope=status,video,photos,friends,wall,notes,groups,docs&response_type=token&v=5.103&state=123456')
        print('После скопируйте ссылку из браузера и вставьте сюда')
        val = str(input())
        self.__get_token_by_link(val) #токен обрабатывается на ошибки в этом методе
        self.__vk_api_on() # подключение VK API к системе


    def __get_token_by_link(self, str):
        arr = str.split('=')
        if(len(arr) >= 2):
            arr2 = arr[1].split('&')
            token = arr2[0]
            print(token)
            self.__token = token
        else:
            print('ссылка повреждена, попробуйте еще раз!')
            print(' ')
            self.init()

    def __vk_api_on(self):
        token = self.__token
        try:
            if(token != None):
                session = vk.Session(token)
                self.api = vk.API(session, v=5.103)

                user = self.api.users.get(user_ids=1) #не работает...

                print('Токен подключен!')
                print(' ')
            else:
                print('Токен отсутствует, попробуйте еще раз!')
                print(' ')
                self.init()
        except Exception:
            print('Токен недействителен или поврежден, попробуйте еще раз')
            print(' ')
            self.init()

In [3]:
c = ConfigAPI()
c.init()

Вставьте ссылку в строку браузера для получения токена VK API. 
https://oauth.vk.com/authorize?client_id=7305133&display=page&scope=status,video,photos,friends,wall,notes,groups,docs&response_type=token&v=5.103&state=123456
После скопируйте ссылку из браузера и вставьте сюда
https://oauth.vk.com/blank.html#access_token=c3270f78eb87520754f9d1f54c2cefdf639fb0c6dadd7d7d33efbb939dfd28244ab78672a45cd57296eaf&expires_in=86400&user_id=599718705&state=123456
c3270f78eb87520754f9d1f54c2cefdf639fb0c6dadd7d7d33efbb939dfd28244ab78672a45cd57296eaf
Токен подключен!
 


In [2]:
gr = c.api.groups.getMembers(group_id = 'kzngo')


NameError: name 'c' is not defined

In [5]:
len(gr['items'])

1000

In [ ]:
gr = c.api.groups.getMembers(group_id = 'kzngo', count = 10)
gr

In [11]:
us = c.api.users.get(user_ids = ['maks_braun', '599718705'])
us

[{'id': 9239006,
  'first_name': 'Макс',
  'last_name': 'Браун',
  'is_closed': False,
  'can_access_closed': True},
 {'id': 599718705,
  'first_name': 'Ne-Fais-Confiance',
  'last_name': 'À-Personne',
  'is_closed': True,
  'can_access_closed': True}]

Данные получить скрапингом, если нужно больше данных. Прогружаем всех пользователей группы, копируем хтмл, выбираем все имена и города

# CHECK SCRAPING
https://www.edureka.co/blog/web-scraping-with-python/

https://www.google.ru/search?q=scraping+python&oq=scraping+python&sourceid=chrome&ie=UTF-8


Скачиваем тыщу человек через апи, берем все уникальные имена и фамилии
Перебираем в поиске всех людей по этим параметрам, скачиваем по тыще человек, добавляем в базу данных

После чистим данные на уникальные значения

Разобраться, как скачать тыщу людей с хтмл

In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [36]:
import requests
names = ['Виталий']
for name in names:
    link = 'https://vk.com/search?c%5Bgroup%5D=38959783&c%5Bname%5D=1&c%5Bper_page%5D=40&c%5Bphoto%5D=1&c%5Bq%5D={}&c%5Bsection%5D=people'.format(name)
    page = requests.get(link)

    soup = BeautifulSoup(page.content, 'html.parser')
    k = soup.find_all('div', class_ = 'labeled')
    print(k)

[]


In [42]:
name = 'Виталий'
link = 'https://vk.com/search?c%5Bgroup%5D=38959783&c%5Bname%5D=1&c%5Bper_page%5D=40&c%5Bphoto%5D=1&c%5Bq%5D={}&c%5Bsection%5D=people'.format(name)
page = requests.get(link)
print(page)
soup = BeautifulSoup(page.content, 'html.parser')
# print(soup)
k = soup.find_all('a', class_ = 'simple_fit_item search_item')
print(len(k))

<Response [200]>
99


In [6]:
with open('labeled name.html', 'r') as f:

    contents = f.read()

    soup = BeautifulSoup(contents, 'html.parser')
    
info = soup.find_all('div', class_ = 'info')

In [7]:
df = pd.DataFrame(columns = ['link', 'name'])

for i in range(len(info)):
    divs = info[i].find_all('div', class_ = 'labeled')
    a_tag = divs[0].find_all('a')
    link = a_tag[0].get('href')
    name = a_tag[0].text
    df = df.append({'link': link, 'name': name}, ignore_index=True)

df

,link,name
0,https://vk.com/yurtoff,Игорь Юртаев
1,https://vk.com/satsura_stas,Стас Сацура
2,https://vk.com/maks_braun,Макс Браун
3,https://vk.com/dinar_shaymardanov,Динар Шаймарданов
4,https://vk.com/barona,Андрей Качерович
...,...,...
994,https://vk.com/id107960712,Алсу Султанова
995,https://vk.com/joseph_adam_jonas_jb,Тимур Неред
996,https://vk.com/didi_didishka,Диана Анисимова
997,https://vk.com/id36845703,Венера Газизова


In [ ]:
names = []
second_name = []
length = len(df)
for i in range(length):
    split = df['name'][i].split(' ')
    names.append(split[0])
    second_name.append(split[1])

names = np.array(names)
second_name = np.array(second_name)

print(len(np.unique(names)))

print(len(np.unique(second_name)))

find methods of classes
how to get text into a tags